# MelSpecClassifier

Use the spectogram of the wav file and use a CNN with 2DConv to classify the genre.
The spectograms have already been provided in the GTZAN dataset. It is cropped before being passed into the model

In [1]:
!pip install "ray[tune]"
import torch
import torch.nn as nn
import torch.optim as optim
import random
import tempfile
from PIL import Image
import torchvision.transforms as transforms
import torch.utils.data as Data
import os
from PIL import ImageOps
from torch.utils.data import SubsetRandomSampler
from ray import train, tune
from ray.train import Checkpoint
from ray.tune.schedulers import ASHAScheduler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.0 MB/s eta 0:00:00


##Mount drive
Mount google drive if running on google colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Constant parameters used in training

Run `setup.sh` to mount Google Drive containing GTZAN

In [3]:
GTZAN_MEL = "/content/drive/MyDrive/GTZAN/Data/images_original/"

PREPROCESS_CROP = (54, 35, 42, 35)

IMAGE_INPUT_DIMENSIONS = [432, 288]
GENRES = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3,
          'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8,
          'rock': 9}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", DEVICE)

Using device cuda


Create a `Dataset` for the mel-spectograms

In [4]:
class ImageDataset(Data.Dataset):
    def __init__(self):
        self.images = []
        self.labels = []

        # Go through all songs and tag X (tensor of image), Y as genre.
        for genre in os.listdir(GTZAN_MEL):
            for song in os.listdir(os.path.join(GTZAN_MEL, genre)):
                abs_path = os.path.join(GTZAN_MEL, genre, song)
                image = Image.open(abs_path)

                # The images have been obtained in the dataset by using the mel spectogram (librosa)
                # Cropping the image to only contain the spectogram to pass into CNN
                image_cropped = ImageOps.crop(image, PREPROCESS_CROP)

                transform = transforms.Compose([transforms.ToTensor()])
                # Convert PIL Image to tensor
                self.images.append(transform(image_cropped))
                # Convert genre tag to associated digit
                self.labels.append(GENRES[genre])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

The `MelSpecTrainer` model used is a CNN with 2 convolutional layers and 2 linear layers.
There is a lack of datapoints compared to the number of dimensions.
To avoid over-training, output features of the linear layer is less, and the number of layers is 2.

In [5]:
class MelSpecTrainer(nn.Module):
    def __init__(self, l1=256, l2=20):
        super().__init__()

        self.current_dimensions = IMAGE_INPUT_DIMENSIONS

        self.conv_layer_1 = nn.Sequential(nn.Conv2d(4, 32, 3),
                                          nn.ReLU(),
                                          nn.MaxPool2d(kernel_size=2, stride=3)
                                          )

        self.conv_layer_2 = nn.Sequential(nn.Conv2d(32, 16, 3),
                                          nn.ReLU(),
                                          nn.MaxPool2d(kernel_size=2, stride=3)
                                          )

        self.flatten_layer = nn.Flatten()

        self.linear_layer_1 = nn.Sequential(nn.Linear(13248, l1),
                                            nn.ReLU())

        self.linear_layer_2 = nn.Sequential(nn.Linear(l1, l2),
                                            nn.ReLU())

        self.classifier = nn.Linear(l2, 10)

    def forward(self, x):
        # First 2D convolution layer
        x = self.conv_layer_1(x)
        # Second 2D convolution layer
        x = self.conv_layer_2(x)

        # Linear layer and classifier
        x = self.flatten_layer(x)
        x = self.linear_layer_1(x)
        x = self.linear_layer_2(x)
        x = self.classifier(x)

        return x

In [6]:
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

Tesla T4


Use the mel spectrogram to train a model for classification of genre of the wav file
Split into test/train/validation.

In [7]:
def dataset_split(image_dataset):
    indices = list(range(len(image_dataset)))
    random.seed(42)
    random.shuffle(indices)

    num_train = int(len(image_dataset) * 0.8)
    num_validation = int(len(image_dataset) * 0.1)
    train_indices = indices[:num_train]
    test_and_validation = indices[num_train:]
    validation_indices = test_and_validation[:num_validation]
    test_indices = test_and_validation[num_validation:]

    return test_indices, train_indices, validation_indices

Create routines for training and validation. Perform Hyperparameter Tuning to devise a closer to optimized model.

In [8]:
def train_mel_spec_model(config):

    model = MelSpecTrainer(l1=config["l1"], l2=config["l2"])
    model.to(DEVICE)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), config["lr"])
    image_dataset = ImageDataset()

    _, train_indices, validation_indices = dataset_split(image_dataset)

    # Create test and train datasets
    train_sampler = SubsetRandomSampler(train_indices)
    validation_sampler = SubsetRandomSampler(validation_indices)

    train_dataset = Data.DataLoader(image_dataset, batch_size=config["batch_size"], sampler=train_sampler)

    validation_dataset = Data.DataLoader(image_dataset, batch_size=config["batch_size"], sampler=validation_sampler)

    for epoch in range(config["num_epochs"]):
        for batch_id, curr_batch in enumerate(train_dataset):
            # Predict and get loss
            images, labels = curr_batch[0].to(DEVICE), curr_batch[1].to(DEVICE)
            pred = model(images)
            loss = loss_fn(pred, labels)

            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            print(f"epoch: {epoch}, batch_id: {batch_id}, loss: {loss}")

        # Validation loss
        # Calculate avg.loss and accuracy for all datapoints in validation set.
        # Based on https://docs.ray.io/en/latest/tune/examples/tune-pytorch-cifar.html
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for data in validation_dataset:
            with torch.no_grad():
                images, labels = data
                images, labels = images.to(DEVICE), labels.to(DEVICE)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = loss_fn(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        # Construct checkpoint for Ray Tuner
        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            path = os.path.join(temp_checkpoint_dir, "checkpoint.pt")
            torch.save(
                (model.state_dict(), optimizer.state_dict()), path
            )
            checkpoint = Checkpoint.from_directory(temp_checkpoint_dir)
            train.report(
                {"loss": (val_loss / val_steps), "accuracy": correct / total},
                checkpoint=checkpoint,
            )

        print(f"Validation Loss: {val_loss / val_steps}, Accuracy: {correct / total}")

Create routine for testing model. The split being used is 80% for training, 10% for validation, and 10% for testing.

In [9]:
def test_mel_spec_model(best_result):
    best_model = MelSpecTrainer(l1=best_result.config["l1"], l2=best_result.config["l2"])
    best_model.to(DEVICE)

    # Get model state from best result
    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")
    model_state, _ = torch.load(checkpoint_path)
    best_model.load_state_dict(model_state)
    image_dataset = ImageDataset()

    # Create test data loader
    test_indices, _, _ = dataset_split(image_dataset)
    test_sampler = SubsetRandomSampler(test_indices)
    test_dataset = Data.DataLoader(image_dataset, batch_size=5, sampler=test_sampler)

    total = 0
    correct = 0
    for data in test_dataset:
        with torch.no_grad():
            images, labels = data
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            outputs = best_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best Model Accuracy {(correct * 100) / total}%")

# Main function

Here, we specify the range for the hyperparameters we want Ray Tune to tune on. Run the training of the model using various hyperparameters.

Test the model using the best trained model as obtained using Ray Tune

In [10]:
def run_mel_spec_classifier():
    config = {
        "l1": tune.choice([k for k in range(100, 1000, 50)]),
        "l2": tune.choice([j for j in range(5, 30, 5)]),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([l for l in range(5, 50, 5)]),
        "num_epochs": tune.choice([25, 35, 45])
    }

    # Only stop trials at least after 20 training iterations
    asha_scheduler = ASHAScheduler(time_attr='training_iteration',
                                   grace_period=20)

    # Adjust resources depending on availability
    tuner = tune.Tuner(tune.with_resources(tune.with_parameters(train_mel_spec_model),
                       resources={"cpu": 2, "gpu": 1}),
                       tune_config=tune.TuneConfig(
                           metric='loss',
                           mode="min",
                           scheduler=asha_scheduler,
                           num_samples=5,
                       ),
                       param_space=config,)

    results = tuner.fit()
    best_result = results.get_best_result("loss", "min")

    test_mel_spec_model(best_result)

run_mel_spec_classifier()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-04-11 03:45:35,566	INFO worker.py:1752 -- Started a local Ray instance.
2024-04-11 03:45:37,226	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2024-04-11 03:45:37,242	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+-----------------------------------------------------------------------------+
| Configuration for experiment     train_mel_spec_model_2024-04-11_03-45-31   |
+-----------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                      |
| Scheduler                        AsyncHyperBandScheduler                    |
| Number of trials                 5                                          |
+-----------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_mel_spec_model_2024-04-11_03-45-31
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-11_03-45-31_719967_229/artifacts/2024-04-11_03-45-37/train_mel_spec_model_2024-04-11_03-45-31/driver_artifacts`

Trial status: 5 PENDING
Current time: 2024-04-11 03:45:38. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerat

(train_mel_spec_model pid=1147) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_mel_spec_model_2024-04-11_03-45-31/train_mel_spec_model_f562d_00000_0_batch_size=30,l1=550,l2=15,lr=0.0002,num_epochs=35_2024-04-11_03-45-37/checkpoint_000000)


(train_mel_spec_model pid=1147) epoch: 1, batch_id: 1, loss: 2.3117690086364746
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 2, loss: 2.2721567153930664
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 3, loss: 2.281172275543213
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 4, loss: 2.2768030166625977
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 5, loss: 2.246859073638916
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 6, loss: 2.2781951427459717
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 7, loss: 2.2893383502960205
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 8, loss: 2.2632343769073486
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 9, loss: 2.2931594848632812
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 10, loss: 2.268843173980713
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 11, loss: 2.280055522918701
(train_mel_spec_model pid=1147) epoch: 1, batch_id: 12, loss: 2.240933895111084
(train_mel_spec_model pid=1147) epoch: 1, 

(train_mel_spec_model pid=1147) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_mel_spec_model_2024-04-11_03-45-31/train_mel_spec_model_f562d_00000_0_batch_size=30,l1=550,l2=15,lr=0.0002,num_epochs=35_2024-04-11_03-45-37/checkpoint_000001)


...

(train_mel_spec_model pid=3674) Validation Loss: 1.8114131718873978, Accuracy: 0.47474747474747475
(train_mel_spec_model pid=3674) epoch: 19, batch_id: 5, loss: 0.7820948362350464 [repeated 173x across cluster]


2024-04-11 03:54:51,004	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_mel_spec_model_2024-04-11_03-45-31' in 0.0154s.



Trial train_mel_spec_model_f562d_00004 completed after 20 iterations at 2024-04-11 03:54:50. Total running time: 9min 13s
+---------------------------------------------------------------------+
| Trial train_mel_spec_model_f562d_00004 result                       |
+---------------------------------------------------------------------+
| checkpoint_dir_name                               checkpoint_000019 |
| time_this_iter_s                                            2.64116 |
| time_total_s                                                73.9089 |
| training_iteration                                               20 |
| accuracy                                                    0.48485 |
| loss                                                        1.76499 |
+---------------------------------------------------------------------+

Trial status: 5 TERMINATED
Current time: 2024-04-11 03:54:51. Total running time: 9min 13s
Logical resource usage: 2.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerat

(train_mel_spec_model pid=3674) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_mel_spec_model_2024-04-11_03-45-31/train_mel_spec_model_f562d_00004_4_batch_size=5,l1=800,l2=10,lr=0.0001,num_epochs=35_2024-04-11_03-45-37/checkpoint_000019)


Best Model Accuracy 46.53465346534654%
